In [1]:
from pathlib import Path
from src.xg_model import XGModel, XGModelNN

In [2]:
model = XGModelNN("../data")
model.fit()

        distance     angle  distance_change  angle_change  time_change  \
461813  0.127822  0.437167         0.285197      0.410787     0.835044   
785084  0.177421  0.795167         0.104817      0.612403     0.813142   
708796  0.671248  0.564278         0.753264      0.444079     0.817248   
891009  0.261322  0.371937         0.211455      0.457988     0.815880   
468814  0.868121  0.520936         0.533323      0.516509     0.814511   
...          ...       ...              ...           ...          ...   
462870  0.796513  0.529046         0.551783      0.560245     0.811088   
721884  0.278686  0.404985         0.575509      0.319919     0.817933   
686331  0.104365  0.500000         0.443006      0.418257     0.814511   
282987  0.312240  0.640563         0.480987      0.477322     0.809719   
690063  0.649150  0.574285         0.751914      0.304754     0.832307   

        prev_event_same_team  is_home  goal_diff  shot_type-Backhand  \
461813                   0.0      1.0  

Fit model


In [2]:
cv = pickle.load(open("../models/logistic_xg.pkl", "rb"))

In [3]:
data = get_data(Path("../"))

data['Outcome'] = np.where(data['Outcome'] == 0, 0, np.where(data['Outcome'] == 1, 0, np.where(data['Outcome'] == 2, 1, 3)))

data = data[data['Outcome'] != 3]

# Convert to lists
features, labels = convert_data(data)

2007
Rink Adjust
Get previous info
Empty nets
Get New Stuff


In [4]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.2, random_state=42)

In [5]:
cv.best_estimator_.score(x_train, y_train)

0.9322800465612572

In [6]:
cv.best_estimator_.score(x_test, y_test)

0.9318071587932875